# EVENT (.TSV) FILES PROCESSING

This file contains the code to process the event files.
It concatenates the standardized event files of the 3 runs and adapts the onset times for smooth transitions between runs.

The resulting file with the entire 'timeline' is saved as: 'sub-control01_task-music_concat_run_events.tsv'

In [2]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ,get_json_from_file

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants
import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json

from IPython.display import display, HTML


Gtk-Message: 11:37:52.697: Failed to load module "canberra-gtk-module"


# 1. Load event files

In [4]:
dataset_id = 'ds000171'
subject = '01'

sample_path = "/home/jovyan/data/dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

event_run1=pd.read_csv(op.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_events.tsv'), sep='\t')
event_run2=pd.read_csv(op.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_run-2_events.tsv'), sep='\t')
event_run3=pd.read_csv(op.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_run-3_events.tsv'), sep='\t')

In [10]:
# Function to standardize and adjust the onset
def standardize_and_adjust_onset(current_run, last_onset=None, last_duration=None):
    current_run['onset'] -= current_run['onset'].min()
    
    if last_onset is not None and last_duration is not None:
        current_run['onset'] += last_onset + last_duration
    return current_run

# Standardizing and adjusting each run
run1_standardized = standardize_and_adjust_onset(event_run1)
last_onset_run1 = run1_standardized['onset'].iloc[-1]  # Get the last onset of run1
last_duration_run1 = run1_standardized['duration'].iloc[-1]  # Get the last duration of run1
run2_standardized = standardize_and_adjust_onset(event_run2, last_onset=last_onset_run1, last_duration=last_duration_run1)

last_onset_run2 = run2_standardized['onset'].iloc[-1]  # Get the last onset of run2
last_duration_run2 = run2_standardized['duration'].iloc[-1]  # Get the last duration of run2
run3_standardized = standardize_and_adjust_onset(event_run3, last_onset=last_onset_run2, last_duration=last_duration_run2)

# Concatenating all runs together
combined_runs = pd.concat([run1_standardized, run2_standardized, run3_standardized], ignore_index=True)

# Define the output path
output_path = os.path.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_concat_run_events.tsv')

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save the combined dataframe to a TSV file
combined_runs.to_csv(output_path, sep='\t', index=False)